In [1]:
cd "./diff_class(class vector fixed)/cat2dog/"

/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Documents/GitHub/latent/pytorch-pretrained-BigGAN/diff_class(class vector fixed)/cat2dog


In [2]:
%load_ext autoreload
%autoreload 2

# target image
import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal, one_hot_from_int)
import logging


logging.basicConfig(level=logging.INFO)

model = BigGAN.from_pretrained('biggan-deep-128')

truncation = 0.5
class_vector = one_hot_from_names(["dog"], batch_size=1)
noise_vector = truncated_noise_sample(truncation=truncation, batch_size=1, seed = 10)

noise_vector = torch.from_numpy(noise_vector)
class_vector = torch.from_numpy(class_vector)


with torch.no_grad():
    output = model(noise_vector, class_vector, truncation)

save_as_images(output, "dog_target")


INFO:pytorch_pretrained_biggan.model:loading model biggan-deep-128 from cache at /Users/xuejiaxin/.pytorch_pretrained_biggan/6371c3777477e4e75187da1b9b526561aac3134f38c7299a3438009ae560e20d.3434ebdfa74a8c17e0e56061cfd905fa163c92f110e88df77b47da6ce9910b48
INFO:pytorch_pretrained_biggan.model:Model config {
  "attention_layer_position": 8,
  "channel_width": 128,
  "class_embed_dim": 128,
  "eps": 0.0001,
  "layers": [
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      16
    ],
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      8
    ],
    [
      false,
      8,
      8
    ],
    [
      true,
      8,
      4
    ],
    [
      false,
      4,
      4
    ],
    [
      true,
      4,
      2
    ],
    [
      false,
      2,
      2
    ],
    [
      true,
      2,
      1
    ]
  ],
  "n_stats": 51,
  "num_classes": 1000,
  "output_dim": 128,
  "z_dim": 128
}

INFO:pytorch_pretrained_biggan.utils:Saving image to dog_target_

In [3]:
from torchvision.models import squeezenet1_0
from tqdm import trange
from torch import nn

DEVICE = 'cpu'
model = model.eval().to(DEVICE)


semantic_model = squeezenet1_0(pretrained=True).to(DEVICE)
semantic_model.classifier = torch.nn.Sequential(
    torch.nn.Flatten()
    )
semantic_model = semantic_model.eval()

soft_max = nn.Softmax(dim = 1)
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)

trunction = 0.5
noise = truncated_noise_sample(truncation=truncation, batch_size=1, seed=10)
noise = torch.nn.Parameter(torch.tensor(noise, requires_grad=True).float().to(DEVICE))
noise_optim = torch.optim.Adam([noise], lr=0.05)

class_vector = one_hot_from_names(['tabby cat'], batch_size=1)
class_vector = torch.from_numpy(class_vector)

L = []
L_pixel = []
L_semantic = []

for iteration in trange(0, 201):
    noise_optim.zero_grad()

    y_hat = model(noise, class_vector, truncation)

    semantic_loss = -((semantic_model(y_hat) - semantic_model(output)) ** 2).mean() ** .5
    #semantic_loss =  -cos_sim(semantic_model(y_hat), semantic_model(output))
    L_semantic.append(semantic_loss.item())

    pixel_loss = abs(y_hat - output).mean()
    L_pixel.append(pixel_loss.item())

    loss = semantic_loss + 30 * pixel_loss
    L.append(loss.item())

    loss.backward()
    noise_optim.step()

    if iteration % 1 == 0:
        save_as_images(y_hat, f"cat2dog_{iteration}")

100%|██████████| 201/201 [09:14<00:00,  2.76s/it]


In [4]:
import matplotlib.pyplot as plt


plt.plot(L)
plt.plot([x*30 for x in L_pixel], 'r')
plt.plot(L_semantic, 'b')

plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
target = y_hat.clone().detach()
save_as_images(target, "target")

noise_fixed = noise.clone().detach()


INFO:pytorch_pretrained_biggan.utils:Saving image to target_0.png


In [8]:
from torchvision.models import squeezenet1_0
from tqdm import trange
from random import randint, seed

DEVICE = 'cpu'
model = model.eval().to(DEVICE)


semantic_model = squeezenet1_0(pretrained=True).to(DEVICE)
semantic_model.classifier = torch.nn.Sequential(
    torch.nn.Flatten()
    )
semantic_model = semantic_model.eval()

soft_max = torch.nn.Softmax(dim = 1)
cos_sim = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

truncation = 0.5

seed(0)
class_find = one_hot_from_int(randint(0,1001))
class_find = torch.nn.Parameter(torch.tensor(class_find, requires_grad=True).float().to(DEVICE))

optim_class = torch.optim.Adam([class_find], lr=0.05)

save_as_images(model(noise_fixed, class_find, truncation), f"start_refind")

L = []
L_pixel = []
L_semantic = []

for iteration in trange(0, 501):
    optim_class.zero_grad()

    #output_1 = soft_max(class_find)
    output_1 = torch.nn.functional.gumbel_softmax(class_find, tau=1, hard=True)

    y_hat = model(noise_fixed, output_1, truncation)

    semantic_loss_1 = -((semantic_model(y_hat) - semantic_model(target)) ** 2).mean() ** .5 
    #semantic_loss_1 = -cos_sim(semantic_model(y_hat), semantic_model(output))
    L_semantic.append(semantic_loss_1.item())

    pixel_loss_1 = abs(y_hat - target).mean()
    L_pixel.append(pixel_loss_1.item())

    loss_1 = semantic_loss_1 + 100 * pixel_loss_1
    L.append(loss_1.item())

    loss_1.backward(retain_graph=True)
    optim_class.step()


    if iteration % 1 == 0:
        print(output_1.argmax())
        save_as_images(y_hat, f"refind_{iteration}")

INFO:pytorch_pretrained_biggan.utils:Saving image to start_refind_0.png
  0%|          | 1/501 [00:04<35:52,  4.30s/it]tensor(765)
INFO:pytorch_pretrained_biggan.utils:Saving image to refind_1_0.png
  0%|          | 2/501 [00:08<35:17,  4.24s/it]tensor(452)
INFO:pytorch_pretrained_biggan.utils:Saving image to refind_2_0.png
  1%|          | 3/501 [00:11<31:40,  3.82s/it]tensor(813)
INFO:pytorch_pretrained_biggan.utils:Saving image to refind_3_0.png
  1%|          | 4/501 [00:13<27:58,  3.38s/it]tensor(613)
INFO:pytorch_pretrained_biggan.utils:Saving image to refind_4_0.png
  1%|          | 5/501 [00:15<25:27,  3.08s/it]tensor(901)
INFO:pytorch_pretrained_biggan.utils:Saving image to refind_5_0.png
  1%|          | 6/501 [00:18<23:45,  2.88s/it]tensor(581)
INFO:pytorch_pretrained_biggan.utils:Saving image to refind_6_0.png
  1%|▏         | 7/501 [00:20<22:20,  2.71s/it]tensor(926)
INFO:pytorch_pretrained_biggan.utils:Saving image to refind_7_0.png
  2%|▏         | 8/501 [00:23<21:41,  2